In [ ]:
# 本篇為透過KMV模型設立dtd(破產距離)

In [ ]:
from scipy.optimize import fsolve
import pandas as pd
import pickle
import time
import datetime
import numpy as np
import statistics
import math
import pandas as pd
from scipy.stats import norm

In [ ]:
# 首先須計算個別公司於6月底時的過去250天股價標準差
# 由於資料龐大 此處僅顯示程式碼 

dsf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\ret 1955_1990.pkl")
funda = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\funda_ibes.pkl")

f_gvk_list = np.unique(funda["gvk"].tolist()).tolist()
permno_list = np.unique(dsf["PERMNO"].tolist()).tolist()
b = [[0,0,0,]]
data = pd.DataFrame(b,columns=[0,1,2])
for i in range(0,2):
    permno = permno_list[i]
    t1 = dsf2[dsf2["PERMNO"]==permno]
    t1.dropna(inplace = True)
    t1.reset_index(inplace = True,drop = True)
    for y in range(1955,2007):
        da = []
        t11 = t1[(t1["year"]==y)&(t1["month"]==6)]
        if t11.empty:
            pass
        else:
            a = t11.index[-1]
            sd = t1.iloc[a-250:a,:]["RET"].std()
            da.append(permno)
            da.append(y)
            da.append(sd)
            daa = [da]
            sd = pd.DataFrame(daa)
            data = pd.concat([data,sd],axis=0)
            
data.to_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\std2.pkl")

In [7]:
funda = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\funda_ibes.pkl")
funda.head()

,gvkey,gvk,sic_str,sic_industry,datadate,fyear,pay_out_ratio,indfmt,consol,popsrc,...,prcc_f,prch_f,prcl_f,adjex_f,rank,au,auop,auopic,ceoso,cfoso
0,1000,001000,nan,na,730.0,1961.0,NaN,INDL,C,D,...,NaN,NaN,NaN,3.341831,NaN,NaN,NaN,NaN,NaN,NaN
1,1000,001000,nan,na,1095.0,1962.0,NaN,INDL,C,D,...,NaN,NaN,NaN,3.341831,NaN,NaN,NaN,NaN,NaN,NaN
2,1000,001000,nan,na,1460.0,1963.0,NaN,INDL,C,D,...,NaN,NaN,NaN,3.244497,NaN,NaN,NaN,NaN,NaN,NaN
3,1000,001000,nan,na,1826.0,1964.0,NaN,INDL,C,D,...,NaN,NaN,NaN,3.089999,NaN,NaN,NaN,NaN,NaN,NaN
4,1000,001000,nan,na,2191.0,1965.0,None,INDL,C,D,...,NaN,NaN,NaN,3.089999,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
std = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\std2.pkl")
std.rename(columns = {0:"permno",1:"year",2:"std",3:"start_prc",4:"end_prc"},inplace=True)

std.dropna(inplace = True)
std.reset_index(inplace = True,drop = True)
std["start_prc"] = std["start_prc"].abs()
std["end_prc"] = std["end_prc"].abs()
std.insert(std.shape[1],"ret",None)
std["ret"] = (std["end_prc"]/std["start_prc"])-1

std.drop([0],inplace = True)
std.head()

,permno,year,std,start_prc,end_prc,ret
1,10000.0,1987,0.048291,2.96875,0.21875,-0.926316
2,10001.0,1987,0.037123,5.93750,5.87500,-0.010526
3,10001.0,1988,0.036748,5.75000,6.25000,0.086957
4,10001.0,1989,0.031856,6.75000,7.00000,0.037037
5,10001.0,1990,0.023518,7.12500,9.75000,0.368421


In [8]:
funda_edf = funda.loc[:,["gvk","fyear","csho","lt","at","ceq"]]
funda_edf.dropna(inplace = True)
funda_edf.reset_index(inplace = True,drop = True)
funda_edf.insert(funda_edf.shape[1],"std",None)
funda_edf.insert(funda_edf.shape[1],"prc",None)
funda_edf.insert(funda_edf.shape[1],"ret",None)

In [9]:
funda_edf.head()

,gvk,fyear,csho,lt,at,ceq,std,prc,ret
0,001000,1964.0,0.196,0.809,1.416,0.607,None,None,None
1,001000,1965.0,0.206,1.818,2.310,0.491,None,None,None
2,001000,1966.0,0.219,1.596,2.430,0.834,None,None,None
3,001000,1967.0,0.219,1.712,2.456,0.744,None,None,None
4,001000,1968.0,0.372,3.351,5.922,2.571,None,None,None


In [11]:
# 以上兩份資料(funda_edf,std)即為計算KMV時所需要的變數
# 然而股價資料來源為CRSP(公司代碼以permno表示)財報資料來源為Compustat(公司代碼以gvkey表示)
# 故需要串接 下面的link可以看到每個年份裡gvkey跟permno(lpermno)是如何對應
link = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\美國資料\link_cik_done.pkl")
link.head()

,gvkey,linkprim,liid,linktype,lpermno,lpermco,USEDFLAG,linkdt,linkenddt,linkdt_day,linkenddt_day,cusip,cik,CIK,y1,y2
0,001000,C,00X,NU,NaN,NaN,-1.0,366.0,3924.0,1961-01-01,1970-09-29 00:00:00,000032102,,0,1961.0,1977.0
1,001000,P,01,NU,NaN,NaN,-1.0,3925.0,3968.0,1970-09-30,1970-11-12 00:00:00,000032102,,0,1961.0,1977.0
2,001000,P,01,LU,25881.0,23369.0,1.0,3969.0,6755.0,1970-11-13,1978-06-30 00:00:00,000032102,,0,1961.0,1977.0
3,001001,C,00X,NU,NaN,NaN,-1.0,6575.0,8662.0,1978-01-01,1983-09-19 00:00:00,000165100,0000723576,723576,1978.0,1985.0
4,001001,P,01,LU,10015.0,6398.0,1.0,8663.0,9708.0,1983-09-20,1986-07-31 00:00:00,000165100,0000723576,723576,1978.0,1985.0


In [15]:
# 由於資料串接過程會有大量缺失值的產生 此處取其中有資料的一部分表現之

problem_gv = []
for i in range(0,len(funda_edf)):

    gv = funda_edf["gvk"][i]
    y = funda_edf["fyear"][i]+1
    t1 = link[link["gvkey"]==gv] 
    if t1.empty:
        pass
    else:
        t1.drop(t1[(t1["linktype"]=="NU") | (t1["linktype"]=="NR")].index,inplace = True)
        t2 = t1[(t1["y1"]<=(y-1))&(t1["y2"]>=y)]
        if len(t2)>2:
            problem_gv.append(gv)
        elif len(t2) == 1:
            permno = t2.iloc[0,4]
            s1 = std[(std["permno"]==permno)&(std["year"]==y)]
            if s1.empty:
                pass
            else:
                funda_edf["std"][i] = s1["std"].values[0]
                funda_edf["prc"][i] = s1["end_prc"].values[0]
                funda_edf["ret"][i] = s1["ret"].values[0]
        else:
            pass

funda_edf.iloc[835:840,:]

<ipython-input-15-cc86dab6d0e9>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funda_edf["std"][i] = s1["std"].values[0]
<ipython-input-15-cc86dab6d0e9>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funda_edf["prc"][i] = s1["end_prc"].values[0]
<ipython-input-15-cc86dab6d0e9>:24: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  funda_edf["ret"][i] = s1["ret"].values[0]


,gvk,fyear,csho,lt,at,ceq,std,prc,ret
835,001054,1984.0,11.655,15.543,51.985,36.442,0.032458,12.75,-0.227273
836,001054,1985.0,12.756,17.725,83.223,65.498,0.032167,11.0,-0.153846
837,001054,1986.0,12.905,23.920,96.689,72.769,0.025702,12.375,0.11236
838,001054,1987.0,13.060,37.822,119.862,82.040,0.0366,15.25,0.232323
839,001054,1988.0,12.927,51.979,143.354,91.375,0.022063,12.5,-0.159664


In [20]:
# dd為kmv模型中的違約距離 edf為違約機率

funda_edf.dropna(inplace = True)
funda_edf.insert(funda_edf.shape[1],"mv",None)
funda_edf.insert(funda_edf.shape[1],"sigma_year",None)
funda_edf.insert(funda_edf.shape[1],"sigma_v",None)
funda_edf.insert(funda_edf.shape[1],"dd",None)
funda_edf.insert(funda_edf.shape[1],"e+f/f",None)
funda_edf.insert(funda_edf.shape[1],"edf",None)


funda_edf["mv"] = funda_edf["prc"]*funda_edf["csho"]
funda_edf["sigma_year"] = funda_edf["std"] * np.sqrt(250)

funda_edf["sigma_v"] = (funda_edf["mv"]/(funda_edf["mv"] + funda_edf["l"]))*funda_edf["sigma_year"]\
   + (funda_edf["l"]/(funda_edf["mv"] + funda_edf["l"]))*(0.05 + 0.25*funda_edf["sigma_year"])

funda_edf["e+f/f"] = (funda_edf["mv"] + funda_edf["l"])/funda_edf["l"]

for i in range(0,len(funda_edf)):
    print(i)
    a = np.log(funda_edf["e+f/f"][i])
    b = funda_edf["ret"][i] - np.square(funda_edf["sigma_v"][i])/2
    dd = (a + b)/funda_edf["sigma_v"][i]
    funda_edf["dd"][i] = dd 
    
funda_edf.insert(funda_edf.shape[1],"edf",None)
N = norm.cdf
for i in range(0,len(funda_edf)):
    print(i)
    a = -funda_edf["dd"][i]
    funda_edf["edf"][i] = N(a, 0.0, 1.0)  
    
funda_edf["edf_100"] = funda_edf["edf"]*100
funda_edf["ceq"] = edf["ceq"].abs()
funda_edf["mtb"] = (edf["csho"]*edf["prc"])/edf["ceq"]
funda_edf["d/e"] = edf["lt"]/edf["ceq"]
funda_edf["a/e"] = edf["at"]/edf["ceq"]

edf = funda_edf.reindex(columns=['gvk',
 'fyear',
 'csho',
 'lt',
 'std',
 'prc',
 'ret',
 'mv',
 'sigma_v',
 'dd',
 'e+f/f',
 'sigma_year',
 'edf',
 'edf_100',
 'at',
 'ceq',
 'mtb',
 'd/e',
 'a/e',
 'ln_a'])
edf.head()

,gvk,fyear,csho,lt,std,prc,ret,mv,sigma_v,dd,e+f/f,sigma_year,edf,edf_100,at,ceq,mtb,d/e,a/e,ln_a
0,001000,1971.0,2.995,20.948,0.044673,6.0,-0.2,17.97,0.448112,0.711901,1.857838,0.706349,0.238263,23.826318,29.33,8.381,2.144136,2.499463,3.499582,3.378611
1,001000,1972.0,2.902,12.886,0.037966,3.0,-0.489362,8.706,0.361445,-0.106521,1.675617,0.600296,0.542416,54.241561,19.907,7.021,1.239994,1.835351,2.835351,2.991071
2,001000,1973.0,2.840,13.204,0.056993,2.625,-0.106383,7.455,0.501133,0.430386,1.564602,0.901145,0.333457,33.34573,21.771,8.567,0.8702,1.541263,2.541263,3.080579
3,001000,1974.0,2.150,15.381,0.044506,2.9375,0.236842,6.315625,0.365004,1.408894,1.410612,0.70371,0.079433,7.943327,25.638,9.843,0.641636,1.562633,2.604694,3.244076
4,001000,1975.0,2.098,11.596,0.041258,7.25,1.636364,15.2105,0.462335,5.12068,2.311702,0.652353,0.0,0.000015,23.905,10.24,1.4854,1.132422,2.334473,3.174088


In [37]:
# edf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\edf_1229.pkl")
# edf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\edf_0121.pkl")
edf = pd.read_pickle(r"C:\Users\user\OneDrive\桌面\金融市場期末報告\edf_drf.pkl")

In [44]:
# 接下來做drf 也就是每一年依edf排序 並且紀錄該筆資料位於該年度edf排序的第幾百分位
# 由於此處有全部資料 但報告只需從1981年開始 故早於1981年的資料會以none呈現

edf.insert(edf.shape[1],"drf",None)

for j in range(1981,2006):
    print(j)
    sample = edf[edf["fyear"]==j]
    ori = [[0,0]]
    data = pd.DataFrame(ori,columns=[0,1])
    if len(sample) < 100:
        pass
    else:
        for i in range(0,len(percent)):
            # print(i)
            li = []
            q = percent[i]
            level = sample["edf"].quantile([q]).values[0]
            li.append(q)
            li.append(level)
            data2 =  pd.DataFrame([li],columns=[0,1])
            data = pd.concat([data,data2],axis=0)
        datay = data.iloc[1:,:]
        datay.rename(columns = {0:"quantile",1:"value"},inplace = True)
        for k in range(0,len(sample)):
            edf_value = sample.iloc[k,34]
            for l in range(0,100):
                # print(l)
                start = datay.iloc[l,1]
                end = datay.iloc[l+1,1]
                if (edf_value >= start) & (edf_value < end):
                    perc = l
                    break
                # else:
                #     pass
            ind = sample.iloc[k].name
            edf["drf"][ind]=perc


,gvk,fyear,csho,lt,std,prc,ret,mv,sigma_v,dd,...,sigma_year,edf,edf_100,at,ceq,mtb,d/e,a/e,ln_a,drf
0,001000,1971.0,2.995,20.948,0.044673,6.0,-0.2,17.97,0.481965,0.920788,...,0.706349,0.17858,NaN,29.33,8.381,2.144136,2.499463,3.499582,3.378611,None
1,001000,1972.0,2.902,12.886,0.037966,3.0,-0.489362,8.706,0.392663,0.228686,...,0.600296,0.409556,NaN,19.907,7.021,1.239994,1.835351,2.835351,2.991071,None
2,001000,1973.0,2.840,13.204,0.056993,2.625,-0.106383,7.455,0.5472,0.573624,...,0.901145,0.283111,NaN,21.771,8.567,0.8702,1.541263,2.541263,3.080579,None
3,001000,1974.0,2.150,15.381,0.044506,2.9375,0.236842,6.315625,0.391754,1.496861,...,0.70371,0.067215,NaN,25.638,9.843,0.641636,1.562633,2.604694,3.244076,None
4,001000,1975.0,2.098,11.596,0.041258,7.25,1.636364,15.2105,0.485683,5.121664,...,0.652353,0.0,NaN,23.905,10.24,1.4854,1.132422,2.334473,3.174088,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
135704,266320,2005.0,70.991,350.574,0.032027,12.96,-0.662588,920.043363,0.44437,2.047323,...,0.506384,0.020313,NaN,553.946,202.951,4.533328,1.727382,2.729457,6.317067,77
135705,269005,2005.0,159.919,3868.173,0.026391,36.549999,0.555319,5845.039328,0.325202,4.771753,...,0.417283,0.000001,NaN,5916.891,2048.718,2.853023,1.888094,2.888094,8.685566,45
135706,270281,2005.0,134.373,1261.775,0.03007,22.389999,-0.100442,3008.611388,0.386252,2.743453,...,0.475443,0.00304,NaN,3600.083,2210.474,1.361071,0.570816,1.628648,8.188712,69
135707,270287,2004.0,48.703,168.221,0.031889,12.89,-0.024962,627.781687,0.434863,3.299449,...,0.504214,0.000484,NaN,403.101,234.424,2.677975,0.717593,1.719538,5.999187,59
